In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# CIFAR 10 is a set of 60K images 32x32 pixels on 3 channels (RGB)
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

# define constants
# EPOCHS = 50
EPOCHS = 10
NUM_CLASSES = 10
# BATCH_SIZE = 128
# VERBOSE = 1
# VALIDATION_SPLIT = 0.2
# OPTIM = tf.keras.optimizers.RMSprop()

In [2]:
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # normalize
    mean = np.mean(x_train, axis=(0,1,2,3))
    std = np.std(x_train, axis=(0,1,2,3))
    x_train = (x_train-mean) / (std+1e-7)
    x_test = (x_test-mean) / (std+1e-7)

    y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

    return x_train, y_train, x_test, y_test

In [3]:
def build_model():
    # We will use 3 modules:
    # 1: CONV + CONV + MaxPool + DropOut
    # 2: CONV + CONV + MaxPool + DropOut
    # 3: CONV + CONV + MaxPool + DropOut
    # After that we apply a pooling operation with pool size 2x2 and a Dropout of 25%
    model = models.Sequential()

    # 1st module
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))

    # 2nd module
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))

    # 3rd module:
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.4))

    # dense:
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

    return model

In [4]:
callbacks = [
    # Write TensorBoard logs to './dcm_cifar_10_logs' directory
    tf.keras.callbacks.TensorBoard(log_dir='./dcm_cifar_10_logs')
]

# Load daya and compile the model:
(x_train, y_train, x_test, y_test) = load_data()

# augment input data
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(x_train)

# build the model
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# train the model
batch_size = 64
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=EPOCHS, verbose=1, validation_data=(x_test, y_test))

# save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')

# test the model
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print("\nTest loss:", score[0])
print("\nTest accuracy:", score[1])

Epoch 1/10
782/782 [==============================] - 129s 164ms/step - loss: 2.5415 - accuracy: 0.2942 - val_loss: 1.4897 - val_accuracy: 0.4914